In [103]:
import zipfile
import pandas as pd
import geopandas as gpd
import os

In [104]:
# extract and read boundary data from zip file
boundaries_zip_path = "../../data/raw/census_boundaries_2011.zip"
z = zipfile.ZipFile(boundaries_zip_path)
z.extractall(path="../../data/raw/") # extract to folder
nhs_boundaries = gpd.read_file("../../data/raw/gct_000b11a_e.shp")

# read 2011 NHS data for BC from zip file
nhs_zip_path = "../../data/raw/nhs_census_2011.zip"
with zipfile.ZipFile(nhs_zip_path,"r") as z:
    with z.open("99-004-XWE2011001-401-BC.csv") as f:
        nhs_2011 = pd.read_csv(f, encoding='latin-1', usecols=[0,2,3,4,5,6,8,10,12])
        
# read in local areas boundaries
areas = gpd.read_file("../../data/raw/local_area_boundary.geojson")

In [105]:
# select only census tracts in Vancouver
van_bound = nhs_boundaries[nhs_boundaries['CMANAME'] == 'Vancouver']

# select and rename needed columns
van_bound = van_bound[['CTUID', 'geometry']]
van_bound.rename(columns={'CTUID':'Geo_Code'}, inplace=True)
van_bound.Geo_Code = van_bound.Geo_Code.apply(lambda x: float(x))
van_bound['LocalArea'] = 'None'
van_bound.reset_index(drop=True, inplace=True)
van_bound

,Geo_Code,geometry,LocalArea
0,9330147.01,"POLYGON ((-123.11399 49.17010, -123.11398 49.1...",None
1,9330187.14,"POLYGON ((-122.76776 49.15303, -122.76776 49.1...",None
2,9330203.00,"POLYGON ((-122.93060 49.20761, -122.93129 49.2...",None
3,9330134.00,"POLYGON ((-123.13981 49.33816, -123.14067 49.3...",None
4,9330403.05,"POLYGON ((-122.63942 49.22053, -122.64141 49.2...",None
...,...,...,...
452,9330188.08,"POLYGON ((-122.76747 49.19152, -122.76827 49.1...",None
453,9330170.03,"POLYGON ((-122.80112 49.03118, -122.80112 49.0...",None
454,9330226.03,"POLYGON ((-122.99420 49.22992, -122.99420 49.2...",None
455,9330147.08,"POLYGON ((-123.12469 49.17019, -123.12469 49.1...",None


In [106]:
# find local area of the census tract based on geom
local_areas= list(van_bound['LocalArea'])

for row in range(len(local_areas)):
    for area in range(len(areas.name)):
        if areas.geometry[area].contains(van_bound.geometry[row].centroid):
            local_areas[row] = str(areas.name[area])
            break

In [107]:
# name local areas based on geom
van_bound['LocalArea'] = local_areas
van_bound = van_bound[van_bound['LocalArea'] != 'None']
van_bound

,Geo_Code,geometry,LocalArea
7,9330028.00,"POLYGON ((-123.12721 49.25712, -123.12723 49.2...",Shaughnessy
14,9330027.01,"POLYGON ((-123.15077 49.25725, -123.15079 49.2...",Arbutus-Ridge
26,9330060.02,"POLYGON ((-123.12918 49.27908, -123.12970 49.2...",West End
28,9330053.02,"POLYGON ((-123.02339 49.28114, -123.02339 49.2...",Hastings-Sunrise
29,9330036.02,"POLYGON ((-123.02353 49.26154, -123.02354 49.2...",Renfrew-Collingwood
...,...,...,...
433,9330015.01,"POLYGON ((-123.02351 49.22923, -123.02351 49.2...",Killarney
434,9330058.00,"POLYGON ((-123.07735 49.28741, -123.07735 49.2...",Strathcona
436,9330004.02,"POLYGON ((-123.09094 49.21827, -123.09097 49.2...",Sunset
437,9330008.01,"POLYGON ((-123.19708 49.23505, -123.19708 49.2...",Dunbar-Southlands


In [108]:
# merge local area to NHS data
merged_df = nhs_2011.merge(van_bound, on = 'Geo_Code')
merged_df = merged_df[['LocalArea', 'CT_Name', 'Topic', 'Characteristic', 'Total', 'Male', 'Female']]
merged_df.Characteristic = merged_df.Characteristic.apply(lambda x: (x.lstrip()).rstrip())

# create a copy of the data frame 
total_df = merged_df.copy()
total_df

,LocalArea,CT_Name,Topic,Characteristic,Total,Male,Female
0,Victoria-Fraserview,14.01,Citizenship,Total population in private households by citi...,3015.0,1395.0,1620.0
1,Victoria-Fraserview,14.01,Citizenship,Canadian citizens,2670.0,1230.0,1440.0
2,Victoria-Fraserview,14.01,Citizenship,Canadian citizens aged under 18,410.0,175.0,235.0
3,Victoria-Fraserview,14.01,Citizenship,Canadian citizens aged 18 and over,2255.0,1050.0,1210.0
4,Victoria-Fraserview,14.01,Citizenship,Not Canadian citizens,345.0,160.0,185.0
...,...,...,...,...,...,...,...
111355,Kitsilano,47.02,Income of individuals in 2010,Prevalence of low income in 2010 based on afte...,20.3,24.2,16.5
111356,Kitsilano,47.02,Income of individuals in 2010,Less than 18 years (%),24.5,32.3,11.1
111357,Kitsilano,47.02,Income of individuals in 2010,Less than 6 years (%),35.7,0.0,25.0
111358,Kitsilano,47.02,Income of individuals in 2010,18 to 64 years (%),20.2,23.8,17.0


In [109]:
# get lists of unique topics and areas
topics = list(total_df['Topic'].unique())
local_areas = list(total_df['LocalArea'].unique())
census_tracts = list(total_df['CT_Name'].unique())

# ensure all variables have a unique name
for ct in census_tracts:
    for topic in topics:
        tract = total_df['CT_Name'] == ct
        tp = total_df['Topic'] == topic
        ind = list(total_df[tract & tp].Characteristic.index)
        for var in range(len(total_df[tract & tp].Characteristic)):
                total_df.Characteristic[ind[var]] = str(var) + '_' + total_df[tract & tp].Characteristic[ind[var]]
        
total_df

/home/keanna/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


,LocalArea,CT_Name,Topic,Characteristic,Total,Male,Female
0,Victoria-Fraserview,14.01,Citizenship,0_Total population in private households by ci...,3015.0,1395.0,1620.0
1,Victoria-Fraserview,14.01,Citizenship,1_Canadian citizens,2670.0,1230.0,1440.0
2,Victoria-Fraserview,14.01,Citizenship,2_Canadian citizens aged under 18,410.0,175.0,235.0
3,Victoria-Fraserview,14.01,Citizenship,3_Canadian citizens aged 18 and over,2255.0,1050.0,1210.0
4,Victoria-Fraserview,14.01,Citizenship,4_Not Canadian citizens,345.0,160.0,185.0
...,...,...,...,...,...,...,...
111355,Kitsilano,47.02,Income of individuals in 2010,106_Prevalence of low income in 2010 based on ...,20.3,24.2,16.5
111356,Kitsilano,47.02,Income of individuals in 2010,107_Less than 18 years (%),24.5,32.3,11.1
111357,Kitsilano,47.02,Income of individuals in 2010,108_Less than 6 years (%),35.7,0.0,25.0
111358,Kitsilano,47.02,Income of individuals in 2010,109_18 to 64 years (%),20.2,23.8,17.0


In [110]:
# split data frame by topics
sub_dataframes = dict()
sub_group = dict()

for topic in topics:
    sub_dataframes[topic] = total_df[total_df['Topic'] == topic]

In [111]:
# Create the nhs directory if it doesn't exist
os.makedirs('../../data/processed/nhs/', exist_ok=True) 


for topic in topics:
    topic_df = sub_dataframes[topic].copy()
    topic_df.drop(columns=['Topic', 'CT_Name'], inplace=True)
    topic_grouped = topic_df.groupby(by=['LocalArea', 'Characteristic']).mean()
    topic_grouped.reset_index(inplace=True)
    sub_group[str(topic)] = pd.DataFrame()
    
    for area in local_areas:
        df = topic_grouped[topic_grouped['LocalArea']==area].copy()
        df.drop(columns=['LocalArea'], inplace=True)
        df = df.set_index('Characteristic').T.reset_index().rename(columns={'index': 'Type'})
        df['LocalArea'] = str(area)
        sub_group[str(topic)] = pd.concat([sub_group[str(topic)], df])
    sub_group[str(topic)].to_csv('../../data/processed/nhs/' + str(topic) + '.csv')

In [112]:
sub_group.keys()

dict_keys(['Citizenship', 'Immigrant status and period of immigration', 'Age at immigration', 'Immigrant status and selected places of birth', 'Recent immigrants by selected place of birth', 'Generation status', 'Visible minority population', 'Ethnic origin population', 'Religion', 'Aboriginal population', 'Non-official languages spoken', 'Mobility', 'Education', 'Language used most often at work', 'Labour force status', 'Class of worker', 'Occupation', 'Industry', 'Work activity', 'Full-time or part-time weeks worked', 'Place of work status', 'Mode of transportation', 'Median commuting duration', 'Time leaving for work', 'Occupied private dwelling characteristics', 'Household characteristics', 'Shelter costs', 'Income of individuals in 2010', 'Income of households in 2010'])